In [1]:
%pip install numpy pandas matplotlib python-pptx

Note: you may need to restart the kernel to use updated packages.


In [2]:
import sys  # noqa
# sys.path.append('/workspace/forecasting_simulation')  # noqa

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import seaborn as sns 
import matplotlib.colors as mcolors
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.dml.color import RGBColor
from utils import * 

FileNotFoundError: [Errno 2] No such file or directory: './assets/Poppins-Regular.ttf'

## LOAD DATA

In [7]:
import pandas as pd

# Load the data
data = pd.read_excel('STA. LUCIA LAND, INC.  (B Centro) AUG 2023 to JUL 2024.xlsx')
data['datetime'] = pd.to_datetime(data['datetime'])  # Ensure datetime is in correct format

# Check the number of columns in the DataFrame
num_columns = data.shape[1]

# Column indices: 8th column (index 7) for kwh and 9th column (index 8) for kw
kwh_column_idx = 7  # 8th column (0-based index)
kw_column_idx = 8   # 9th column (0-based index)

# Check datetime interval to determine if it's 5-minute or 1-hour data
time_diff = data['datetime'].diff().dt.total_seconds().median()
frequency = 12 if time_diff == 300 else 1  # 5-minute interval if median diff is 300 seconds, otherwise 1 hour

# Handling 5-minute data
if frequency == 12:  # 5-minute interval
    print("Data is 5-minute interval.")

    # Check if there are 8 or 9 columns
    if num_columns == 8:
        # Check if the 8th column is named as 'kw'
        if data.columns[kwh_column_idx].lower() == 'kw':
            # Rename it to 'kwh' and create 'kw' as the 9th column
            data.rename(columns={data.columns[kwh_column_idx]: 'kwh'}, inplace=True)
            data['kw'] = data['kwh'] * 12  # Create the 9th column by multiplying kwh by 12
        else:
            # If it's already kwh, just create 'kw' column
            data['kw'] = data.iloc[:, kwh_column_idx] * 12  # Create the 9th column by multiplying kwh by 12
            data.rename(columns={data.columns[kwh_column_idx]: 'kwh'}, inplace=True)
            
        # Ensure the data has 9 columns now
        print(f"Data now has {data.shape[1]} columns (9 columns expected).")

    elif num_columns == 9:
        # Simply rename the 8th and 9th columns as kwh and kw
        data.rename(columns={data.columns[kwh_column_idx]: 'kwh', data.columns[kw_column_idx]: 'kw'}, inplace=True)

# Handling 1-hour data
elif frequency == 1:  # 1-hour interval
    print("Data is 1-hour interval.")

    # Check if there are 8 or 9 columns
    if num_columns == 8:
        # Copy the value from the 8th column (kwh) to the 9th column (kw)
        data['kw'] = data.iloc[:, kwh_column_idx]
        # Rename the columns appropriately
        data.rename(columns={data.columns[kwh_column_idx]: 'kwh', 'kw': 'kw'}, inplace=True)
        print(f"Data now has {data.shape[1]} columns (9 columns expected).")

    elif num_columns == 9:
        # Ensure the 8th and 9th columns are the same
        if not (data.iloc[:, kwh_column_idx] == data.iloc[:, kw_column_idx]).all():
            print("Warning: 8th and 9th columns do not match.")
        # Rename the columns as kwh and kw
        data.rename(columns={data.columns[kwh_column_idx]: 'kwh', data.columns[kw_column_idx]: 'kw'}, inplace=True)

# Show first few rows to verify the results
print(data.head())


Data is 1-hour interval.
Data now has 9 columns (9 columns expected).
  supply period            datetime       date  correct hour  hour  weekday  \
0        Aug-23 2023-07-26 00:00:00 2023-07-26             0     1        2   
1        Aug-23 2023-07-26 01:00:00 2023-07-26             1     2        2   
2        Aug-23 2023-07-26 02:00:00 2023-07-26             2     3        2   
3        Aug-23 2023-07-26 03:00:00 2023-07-26             3     4        2   
4        Aug-23 2023-07-26 04:00:00 2023-07-26             4     5        2   

       wesm      kwh       kw  
0  3.937709  114.100  114.100  
1  3.725969  112.525  112.525  
2  3.140275  110.250  110.250  
3  2.408305  111.825  111.825  
4  2.418903  112.175  112.175  


In [10]:
# Check the columns in your data
print(data.columns)

Index(['supply period', 'datetime', 'date', 'correct hour', 'hour', 'weekday',
       'wesm', 'kwh', 'kw'],
      dtype='object')


In [11]:
# Generate energy summary
energy_summary = generate_energy_summary(data, kwh_column='kwh', kw_column='kw')
energy_summary

NameError: name 'generate_energy_summary' is not defined